In [1]:
from gpt.gpt_distgen import run_gpt_with_distgen, evaluate_gpt_with_distgen
from gpt import GPT
import yaml
import numpy as np

def test_cu_injector_template():

    settings={'n_particle':100, 
          #'final_n_particle':200,
          'r_dist:sigma_xy:value':3,
          'Ntout':100,
          'gun_voltage':400,
          'buncher_voltage':50,
          'phiCTB01_off':-90, 
          'cavity_1_voltage':1000,
          'cavity_2_voltage':1000,
          'cavity_3_voltage':1000,
          'cavity_4_voltage':1000,
          'cavity_5_voltage':1000,
          'A3Quad_1_current':3,
          'A3Quad_2_current':-3,
          'A3Quad_3_current':3
         }

    G = run_gpt_with_distgen(settings,
                         gpt_input_file='../examples/templates/cu_injector/gpt.in',
                         distgen_input_file='../examples/templates/cu_injector/distgen.in.yaml',
                         verbose=True,
                         auto_phase=True,
                         parse_layout=False
                        )
                         #workdir='templates/cu_injector/temp',
                         #use_tempdir=False)

    G1 = GPT()
    G1.load_archive('../examples/templates/cu_injector/test.gpt.h5')

    assert G.n_screen == G1.n_screen, 'Inconsistent number of screens'
    assert G.n_tout == G1.n_tout, 'Inconsistent number of touts'

    keys = ['x', 'y', 'z', 'px', 'pz', 'py', 't', 'id']
    
    for ii, scr in enumerate(G.screen):

        for key in keys:
            assert np.all(np.isclose(scr[key], G1.screen[ii][key], equal_nan=True))

    for ii, tout in enumerate(G.tout):

        for key in keys:
            assert np.all(np.isclose(tout[key], G1.tout[ii][key]))

    with open('../examples/templates/cu_injector/eval.test.yaml', 'r') as fid:
        res1 = yaml.safe_load(fid)

    res = evaluate_gpt_with_distgen(settings=settings,
                                    gpt_input_file='../examples/templates/cu_injector/gpt.in',
                                    distgen_input_file='../examples/templates/cu_injector/distgen.in.yaml',
                                    verbose=True,
                                    auto_phase=True,
                                    parse_layout=False
                                   )

    print('\n')
    for k in res:
        if isinstance(res[k], float):
            assert np.isclose(res[k] ,res1[k]), f'Bad eval merit: {k}' 
            print(k, 'checks')




In [2]:
test_cu_injector_template()

Run GPT with Distgen:
n_particle is in distgen
Replacing param n_particle with value 100
r_dist:sigma_xy:value is in distgen
Replacing param r_dist:sigma_xy:value with value 3
Ntout is in gpt
gun_voltage is in gpt
buncher_voltage is in gpt
phiCTB01_off is in gpt
cavity_1_voltage is in gpt
cavity_2_voltage is in gpt
cavity_3_voltage is in gpt
cavity_4_voltage is in gpt
cavity_5_voltage is in gpt
A3Quad_1_current is in gpt
A3Quad_2_current is in gpt
A3Quad_3_current is in gpt
Linking particle files, distgen output will point to -> "gpt_particles.gdf" in working directory.
Replacing Distgen output params

Distgen >------

Distribution format: gpt
Output file: /var/folders/w8/8n1md1352pzd3nqp0lhc5jy00000gn/T/tmpqx1y8spg/gpt_particles.gdf

Creating beam distribution....
   Beam starting from: cathode
   Total charge: 1 nC.
   Number of macroparticles: 100.
   Assuming cylindrical symmetry...
   r distribution: SuperGaussianRad
      lambda = 4.24264 mm, power = INF
   theta distribution: un